# 词嵌入（Word Embedding）基础

## 1. 从One-Hot到词嵌入的演进

### 1.1 One-Hot编码的局限性

回顾One-Hot编码的问题：

```python
"cat"  -> [0, 0, 1, 0, 0, 0, ..., 0]  # 10,000维，只有1个非零
"dog"  -> [0, 0, 0, 0, 1, 0, ..., 0]  # 10,000维，只有1个非零
"kitten" -> [0, 1, 0, 0, 0, 0, ..., 0]  # 10,000维，只有1个非零
```

**核心问题**：
1. **维度灾难**：词汇表10,000个词 = 10,000维向量
2. **语义缺失**："cat"和"kitten"语义相近，但向量完全正交（内积=0）
3. **稀疏性**：99.99%的元素都是0，浪费存储和计算
4. **泛化能力差**：模型无法利用词之间的相似性

### 1.2 词嵌入的核心思想

**词嵌入（Word Embedding）**将高维稀疏的One-Hot向量映射到低维稠密的连续向量空间：

```python
"cat"    -> [0.2, -0.4, 0.7, ..., 0.1]  # 300维，全是实数
"dog"    -> [0.3, -0.3, 0.8, ..., 0.2]  # 与cat相近
"kitten" -> [0.25, -0.35, 0.65, ..., 0.15]  # 更接近cat
"car"    -> [-0.8, 0.6, -0.2, ..., 0.9]  # 距离cat很远
```

**关键特性**：
1. **低维稠密**：通常为100-300维，远小于词汇表大小
2. **语义相似性**：相似词在向量空间中距离近
3. **可学习**：向量值通过训练学习得到
4. **分布式表示**：每个维度捕捉词的某个语义特征

## 2. 词嵌入的理论基础

### 2.1 分布式假设（Distributional Hypothesis）

> "You shall know a word by the company it keeps."  
> — J.R. Firth (1957)

**核心观点**：词的含义由其上下文决定。

**例子**：
- "猫"经常出现在："宠物", "喵", "抓老鼠", "猫粮" 附近
- "狗"经常出现在："宠物", "汪", "看家", "狗粮" 附近
- 因此"猫"和"狗"应该有相似的向量表示

### 2.2 词嵌入的数学本质

词嵌入本质上是一个**查找表（Lookup Table）**：

```
Embedding矩阵: E ∈ R^(V × d)
- V: 词汇表大小（如10,000）
- d: 嵌入维度（如300）

查找操作:
word_id = 42  # "cat"的索引
word_vector = E[42, :]  # 取出第42行，得到300维向量
```

**等价视角**：
```
One-Hot × Embedding矩阵 = 词向量
[0, 0, 1, ..., 0] × E = E的第3行
    (V维)         (V×d)   (d维)
```

### 2.3 词嵌入的几何解释

在嵌入空间中：
- **距离**：相似词距离近（欧氏距离或余弦距离）
- **方向**：捕捉语义关系
- **类比**：向量运算可表示类比关系

**著名例子**：
```
King - Man + Woman ≈ Queen
Paris - France + Italy ≈ Rome
```

## 3. Keras中的Embedding层

Keras提供了`Embedding`层来实现词嵌入。

### 3.1 Embedding层的参数

```python
Embedding(input_dim, output_dim, input_length)
```

- **input_dim**：词汇表大小（整数索引的最大值+1）
- **output_dim**：嵌入维度（词向量的长度）
- **input_length**：输入序列的长度（固定长度序列）

### 3.2 Embedding层的工作流程

```
输入：整数序列  [5, 12, 3, 8]        形状: (batch_size, seq_length)
       ↓
Embedding层：查找嵌入矩阵
       ↓
输出：词向量序列  [[v5], [v12], [v3], [v8]]  形状: (batch_size, seq_length, embedding_dim)
```

### 3.3 关键特性

1. **可训练**：嵌入矩阵的权重可以通过反向传播学习
2. **参数共享**：相同的词总是映射到相同的向量
3. **高效**：只是查找操作，不涉及矩阵乘法

## 4. 实践：在IMDB数据集上使用词嵌入

### 4.1 任务说明

- **数据集**：IMDB电影评论情感分类
- **任务**：根据评论文本判断情感（正面/负面）
- **方法**：使用Embedding层 + 全连接层

### 4.2 数据预处理

In [ ]:
from tensorflow.keras import preprocessing
from tensorflow.keras.datasets import imdb

# 设置超参数
max_features = 10000  # 词汇表大小：只考虑最常出现的10,000个单词
maxlen = 20          # 序列最大长度：每条评论只取前20个单词

print("=" * 60)
print("加载IMDB数据集")
print("=" * 60)

# 加载数据集
# num_words参数确保只保留最常见的max_features个词
# 每个词被映射为一个整数索引（按词频排序）
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(f"训练集大小: {len(x_train)}")
print(f"测试集大小: {len(x_test)}")
print(f"\n第一条评论（整数序列）: {x_train[0][:10]}...")  # 只显示前10个词
print(f"第一条评论长度: {len(x_train[0])}")
print(f"第一条评论标签: {y_train[0]} ({'正面' if y_train[0] == 1 else '负面'})")

print("\n" + "=" * 60)
print("序列填充")
print("=" * 60)

# 序列填充：将所有序列统一为相同长度
# - 短于maxlen的序列：在前面填充0
# - 长于maxlen的序列：截断
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

print(f"填充后的训练集形状: {x_train.shape}")
print(f"填充后的测试集形状: {x_test.shape}")
print(f"\n第一条评论（填充后）: {x_train[0]}")
print(f"\n解释：")
print(f"  - 形状 (25000, 20) 表示 25,000条评论，每条20个词")
print(f"  - 0表示填充位置（padding）")
print(f"  - 非零数字是词的索引（1-9999）")

### 4.3 模型架构

我们构建一个简单的模型：

```
输入: (batch_size, 20)  # 20个词的整数序列
   ↓
Embedding: (batch_size, 20, 8)  # 每个词变成8维向量
   ↓
Flatten: (batch_size, 160)  # 展平为一维向量
   ↓
Dense: (batch_size, 1)  # 输出单个概率值
```

**设计说明**：
1. **Embedding维度选择8**：通常为50-300，这里为演示用较小值
2. **Flatten层**：将3D张量展平为2D，方便全连接层处理
3. **无循环结构**：这个模型不考虑词序，类似bag-of-words

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

print("=" * 60)
print("构建模型")
print("=" * 60)

model = Sequential([
    # Embedding层：将整数索引映射为稠密向量
    # input_dim=10000: 词汇表大小
    # output_dim=8: 每个词的嵌入维度
    # input_length=20: 输入序列长度
    Embedding(input_dim=max_features, output_dim=8, input_length=maxlen, name='embedding'),
    
    # Flatten层：将3D张量(batch, 20, 8)展平为2D张量(batch, 160)
    Flatten(name='flatten'),
    
    # 全连接层：输出单个概率值用于二分类
    Dense(1, activation='sigmoid', name='output')
])

# 编译模型
model.compile(
    optimizer='adam',                # Adam优化器
    loss='binary_crossentropy',      # 二分类交叉熵损失
    metrics=['accuracy']              # 监控准确率
)

# 显示模型结构
model.summary()

print("\n模型参数分析：")
print(f"  Embedding层参数: {max_features} × 8 = {max_features * 8:,}")
print(f"  Dense层参数: 160 × 1 + 1(bias) = 161")
print(f"  总参数: {max_features * 8 + 161:,}")

### 4.4 训练模型

训练过程中，Embedding层的权重会自动学习，使得：
- 有助于正面情感的词向量朝一个方向移动
- 有助于负面情感的词向量朝另一个方向移动
- 语义相近的词向量距离变近

In [ ]:
print("=" * 60)
print("训练模型")
print("=" * 60)

history = model.fit(
    x_train, y_train,
    epochs=10,                    # 训练10个epoch
    batch_size=32,                # 批次大小32
    validation_split=0.2          # 20%数据用于验证
)

print("\n训练完成！")

### 4.5 分析学习到的词嵌入

训练完成后，我们可以提取和分析学到的词向量。

In [ ]:
import numpy as np

print("=" * 60)
print("提取词嵌入矩阵")
print("=" * 60)

# 获取Embedding层的权重（即词嵌入矩阵）
embedding_layer = model.get_layer('embedding')
embedding_weights = embedding_layer.get_weights()[0]

print(f"嵌入矩阵形状: {embedding_weights.shape}")
print(f"  - {embedding_weights.shape[0]} 个词")
print(f"  - 每个词 {embedding_weights.shape[1]} 维向量")

# 查看几个词的嵌入向量
print("\n示例词向量：")
for word_id in [1, 2, 3, 4, 5]:  # IMDB中1="the", 2="and", 3="a", 4="of", 5="to"
    word_vector = embedding_weights[word_id]
    print(f"  词索引 {word_id}: {word_vector}")

# 计算词向量之间的余弦相似度
def cosine_similarity(v1, v2):
    """计算两个向量的余弦相似度"""
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

print("\n词向量相似度分析：")
vec1 = embedding_weights[1]  # "the"
vec2 = embedding_weights[2]  # "and"
vec3 = embedding_weights[3]  # "a"

print(f"  词1(the) vs 词2(and): {cosine_similarity(vec1, vec2):.4f}")
print(f"  词1(the) vs 词3(a):   {cosine_similarity(vec1, vec3):.4f}")
print(f"  词2(and) vs 词3(a):   {cosine_similarity(vec2, vec3):.4f}")

print("\n注意：")
print("  - 相似度在[-1, 1]之间")
print("  - 值越接近1，词越相似")
print("  - 功能词（the, and, a）通常具有相似的向量")

## 5. 词嵌入的两种学习方式

### 5.1 任务相关的嵌入（Task-Specific Embedding）

**特点**：
- 随机初始化嵌入矩阵
- 在特定任务上从头训练
- 嵌入向量针对当前任务优化

**优势**：
- 简单直接，无需额外数据
- 嵌入表示与任务完美匹配

**劣势**：
- 需要大量标注数据
- 训练时间长
- 无法利用外部语言知识

**适用场景**：
- 有大量任务相关数据
- 领域特殊，通用词向量不适用

### 5.2 预训练嵌入（Pre-trained Embedding）

**特点**：
- 使用在大规模语料上预训练的词向量
- 常见的预训练模型：Word2Vec, GloVe, FastText
- 可以固定或微调

**优势**：
- 包含丰富的语言知识
- 数据量小时效果更好
- 训练速度快

**劣势**：
- 可能不完全适配特定任务
- 需要下载和管理大文件

**适用场景**：
- 标注数据有限
- 通用NLP任务
- 需要快速原型

## 6. Embedding层 vs 全连接层

### 从数学角度看，Embedding等价于特殊的全连接层：

```python
# Embedding层的实现
output = embedding_matrix[word_id]  # 直接查找

# 等价于全连接层的实现
one_hot = to_one_hot(word_id, vocab_size)  # [0, 0, 1, 0, ...]
output = one_hot @ embedding_matrix  # 矩阵乘法
```

### 为什么使用Embedding层而不是全连接层？

1. **效率**：
   - Embedding：O(1)查找操作
   - 全连接：O(V)矩阵乘法，V为词汇表大小

2. **内存**：
   - Embedding：只存储嵌入矩阵
   - 全连接：需要先创建One-Hot向量（稀疏且大）

3. **语义**：
   - Embedding：明确表达"词查找"的语义
   - 全连接：通用层，语义不明确

## 7. 实践建议

### 7.1 嵌入维度选择

| 词汇表大小 | 推荐维度 | 说明 |
|----------|---------|------|
| < 10K | 50-100 | 小词汇表，低维度足够 |
| 10K-100K | 100-300 | 中等词汇表，标准选择 |
| > 100K | 300-500 | 大词汇表，需要更高维度 |

**经验法则**：embedding_dim ≈ 4 × log2(vocab_size)

### 7.2 训练策略

1. **数据充足**（>100K样本）：
   - 从头训练词嵌入
   - 或使用预训练嵌入并微调

2. **数据有限**（<10K样本）：
   - 使用预训练嵌入
   - 冻结嵌入层（不训练）

3. **数据中等**（10K-100K样本）：
   - 使用预训练嵌入
   - 微调嵌入层（较小学习率）

### 7.3 常见陷阱

1. **嵌入维度过高**：
   - 增加过拟合风险
   - 训练速度变慢
   - 收益递减

2. **忽略padding**：
   - 索引0应该保留给padding
   - 设置`mask_zero=True`可以忽略padding位置

3. **不考虑OOV**：
   - 预留特殊索引给未知词（UNK）
   - 或使用字符级/子词级模型

## 8. 总结

### 核心要点：

1. **词嵌入解决了One-Hot编码的主要问题**：
   - 降维：10,000维 → 300维
   - 捕捉语义：相似词距离近
   - 可学习：通过数据自动学习

2. **Keras的Embedding层**：
   - 本质是可训练的查找表
   - 输入整数序列，输出稠密向量序列
   - 可以随机初始化或加载预训练权重

3. **下一步学习**：
   - 预训练词向量（Word2Vec, GloVe, FastText）
   - 上下文嵌入（ELMo, BERT, GPT）
   - 子词级模型（BPE, WordPiece）